## Image Classification 
- Use Google Colab for the tutorial


### Key features of FastAI for image classification
- Learning rate finder (`lr_find`)
- Progressive resizing
- Differential learning rates during fine-tuning
- Powerful visualization:
    - confusion matrix
    - top losses analysis
    - Model export/import with `.pkl` files

You will get practice on all these features from this tutorial

### Step 1: Setup Google Colab environment

In [ ]:
!pip install fastai --upgrade
!pip install fastcore
!pip install ipywidgets
!jupyter nbextension enable --py widgetsnbextension


### Step 2: Import necessary libraries

In [ ]:
from fastai.vision.all import *
from fastai.callback.all import *
from fastai.imports import *
import pathlib

### Step 3: Mount Google Drive to access the dataset

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


### Step 4: Define dataset path (modify according to your folder structure)

#### Dataset Structure
Organize your images in this structure:
   ```
   poultry_dataset/
       cocci/
           image1.jpg
           image2.jpg
           ...
       healthy/
           image1.jpg
           image2.jpg
           ...
       ncd/
           image1.jpg
           image2.jpg
           ...
       other/
           image1.jpg
           image2.jpg
           ...
       salmo/
           image1.jpg
           image2.jpg
           ...
   ```

In [ ]:
path = Path('/content/drive/MyDrive/poultry_dataset')  # Change to your dataset path
classes = ['cocci', 'healthy', 'ncd', 'other', 'salmo']  # disease and healthy poultry classes


### Step 5: Prepare DataBlock

In [ ]:
poultry = DataBlock(
    blocks=(ImageBlock, CategoryBlock),  # Input: images, Output: categories
    get_items=get_image_files,          # Get all image files
    splitter=RandomSplitter(valid_pct=0.2, seed=42),  # 80-20 train-valid split
    get_y=parent_label,                 # Get labels from folder names
    item_tfms=Resize(460),              # Initial resize
    batch_tfms=aug_transforms(size=224, min_scale=0.75)  # Augmentations
)


### Step 6: Create DataLoaders

In [ ]:
dls = poultry.dataloaders(path, bs=32)  # Batch size of 32


### Step 7: Verify data

In [ ]:
dls.show_batch(max_n=6, nrows=2)  # Show sample batch


### Step 8: Create Learner with pretrained ResNet34

In [ ]:
learn = vision_learner(
    dls,
    resnet34,               # Pretrained architecture
    metrics=[accuracy, error_rate],  # Track accuracy and error rate
    model_dir='/tmp/model'  # Directory to save models
)


### Step 9: Find optimal learning rate

In [ ]:
lr_min, lr_steep = learn.lr_find(suggest_funcs=(minimum, steep))
print(f"Minimum/10: {lr_min:.2e}, steepest point: {lr_steep:.2e}")


### Step 10: Train model (fine-tuning)

In [ ]:
learn.fine_tune(
    epochs=10,
    base_lr=lr_steep,       # Use the suggested learning rate
    freeze_epochs=2,        # First train only new layers
    cbs=[ShowGraphCallback(), SaveModelCallback()]  # Show graph and save best model
)


### Step 11: Evaluate model

In [ ]:
interp = ClassificationInterpretation.from_learner(learn)
interp.plot_confusion_matrix(figsize=(8,8), dpi=60)


### Step 12: Show top losses

In [ ]:
interp.plot_top_losses(6, nrows=2)


### Step 13: Export model

In [ ]:
learn.export('poultry_classifier.pkl')


### Step 14: Create prediction function

In [ ]:
def predict_poultry(img_path):
    img = PILImage.create(img_path)
    pred, _, probs = learn.predict(img)
    print(f"Prediction: {pred}")
    print(f"Probabilities: {dict(zip(classes, map(float, probs)))}")
    return img.show()


### Step 15: Gradio demo - Prototype

Gradio is an open-source Python library designed to make it easy to demo, share, and test models with minimal code

In [ ]:
!pip install gradio
import gradio as gr


In [ ]:
def classify_image(inp):
    pred, _, probs = learn.predict(inp)
    return {classes[i]: float(probs[i]) for i in range(len(classes))}


In [ ]:
demo = gr.Interface(
    fn=classify_image,
    inputs=gr.Image(),
    outputs=gr.Label(num_top_classes=3),
    examples=[str(f) for f in (path/'testimg').ls()[:2]]
)


In [ ]:
demo.launch(share=True)

## TO DO
- Change `resnet34` to other architectures like:
     - `resnet18` (faster, less accurate)
     - `resnet50` (slower, more accurate)
- Compare the model performance from the various architectures

### Reference 
[Practical Deep Learning for Coders](https://course.fast.ai/)